## Welcome to the NClimGrid notebook!

#### **Intent:** This notebook will find and visualize one day of NClimGrid data for CONUS, then compare it to historical averages.    
#### **Audience:** Anyone with at least 4GB of memory on their computer or computing environment. No programming experience required to go through this notebook, but it will help you understand where this data is coming from.     
#### **Outcome:** Plots depicting NClimGrid data from the selected day, historical record, and a comparison of the two.        

The NOAA Monthly U.S. Climate Gridded Dataset (NClimGrid) consists of four climate variables derived from the GHCN-D dataset: maximum temperature, minimum temperature, average temperature and precipitation. Each file provides monthly values in a 5x5 lat/lon grid for the Continental United States. Data is available from 1895 to the present (NCEI).

This notebook will use NClimGrid data to plot climate variables across the U.S. for a selected day. Then, it will compare those variables to the climate average.

### Step 1: Import Python modules

In [ ]:
import s3fs
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import netCDF4
import requests
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy import feature as cf
from tqdm.notebook import tqdm
from time import sleep

import warnings
warnings.filterwarnings('ignore')

### Step 2: Review the data available on the AWS cloud, then select the date for your analysis.
Goal: select the date to analyze and retrieve the data. 

In [ ]:
# Formatting settings for drop-down menus
style = {'description_width':'120px'}
layout = widgets.Layout(width='325px')

# Create drop-down menus using widgets, pulling aws data so I don't have to hard code anything
aws = s3fs.S3FileSystem(anon=True)
dirs = aws.ls('noaa-nclimgrid-daily-pds/v1-0-0/grids/', refresh=True)  
    
# Create list of subsetted file names that fall within specified time period(s)
data = []
years = []

for file in dirs:
    year = file[38:]
    years.append(year)

month_options = widgets.Dropdown(options=[('Jan', '01'), ('Feb', '02'), ('Mar', '03'), ('Apr', '04'), ('May', '05'), ('Jun', '06'), ('Jul', '07'), ('Aug', '08'), ('Sep', '09'), ('Oct', '10'), ('Nov', '11'), ('Dec', '12')], description='Month:', style=style, layout=layout)
day_options = widgets.Dropdown(options=[('01'), ('02'), ('03'), ('04'), ('05'), ('06'), ('07'), ('08'), ('09'), ('10'), ('11'), ('12'), ('13'), ('14'), ('15'), ('16'), ('17'), ('18'), ('19'), ('20'), ('21'), ('22'), ('23'), ('24'), ('25'), ('26'), ('27'), ('28'), ('29'), ('30'), ('31')], description='Day:', style=style, layout=layout)
year_options = widgets.Dropdown(options= [s for s in years], description='Year:', style=style, layout=layout)

# Display drop-down menus
print('Select your desired month/day/year, then run the next block. Do NOT re-run this block, or your values will reset!')
display(month_options)
display(day_options)
display(year_options)

### Step 3: Plot the data from your chosen date. 
Goal: visualize all four variables for your chosen day within the entire continental U.S.

In [ ]:
#getting variables from AWS
resp = requests.get(f'https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/v1-0-0/grids/{year_options.value}/ncdd-{year_options.value}{month_options.value}-grd-scaled.nc')
file_name = f'ncdd-{year_options.value}{month_options.value}-grd-scaled.nc'
nc = netCDF4.Dataset(file_name, memory = resp.content)

#extracting variables from netcdf file
tmax = nc.variables["tmax"][:] 
tmax_units = nc.variables["tmax"].units 

tmin = nc.variables["tmin"][:] 
tmin_units = nc.variables["tmin"].units 

prcp = nc.variables["prcp"][:] 
prcp_units = nc.variables["prcp"].units 

tavg  = nc.variables["tavg"][:] 
tavg_units  = nc.variables["tavg"].units
        
lats = nc.variables['lat'][:]
lons = nc.variables['lon'][:]

nc.close()

day_value = int(day_options.value)-1

#plotting
fig, axs = plt.subplots(nrows=2,ncols=2, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(16,9))

pc1 = axs[0,0].pcolormesh(lons, lats, tmax[day_value], cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc1, ax=axs[0,0],fraction=0.026, pad=0.04)
axs[0,0].set_title(f"Maximum temperature on {month_options.value}/{day_options.value}/{year_options.value}")

pc2 = axs[0,1].pcolormesh(lons, lats, tmin[day_value], cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc2, ax=axs[0,1],fraction=0.026, pad=0.04)
axs[0,1].set_title(f"Minimum temperature on {month_options.value}/{day_options.value}/{year_options.value}")

pc3 = axs[1,0].pcolormesh(lons, lats, tavg[day_value], cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc3, ax=axs[1,0],fraction=0.026, pad=0.04)
axs[1,0].set_title(f"Average temperature on {month_options.value}/{day_options.value}/{year_options.value}")

pc4 = axs[1,1].pcolormesh(lons, lats, prcp[day_value], cmap='jet', vmin=0, vmax=50)
plt.colorbar(pc4, ax=axs[1,1],fraction=0.026, pad=0.04)
axs[1,1].set_title(f"Precipitation amount on {month_options.value}/{day_options.value}/{year_options.value}")

#formatting the plots
for i in range(0,2):
    for j in range(0,2):
        axs[i,j].add_feature(cf.LAKES)
        axs[i,j].add_feature(cf.OCEAN)
        states = cf.NaturalEarthFeature(
                category='cultural',
                name='admin_1_states_provinces_lines',
                scale='50m',
                facecolor='none')
        axs[i,j].add_feature(states, edgecolor="white")
        gl = axs[i,j].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', alpha=0.05, linestyle='-')
        gl.top_labels = False
        gl.left_labels = True
        gl.right_labels = False
        gl.xlines = True
        gl.ylines = True
plt.show(fig)
plt.close(fig)
plt.close("all")

### Step 4: Calculate the climatological average for your day.
Goal: visualize the typical temperature and precipitation conditions for your chosen day by taking the average of historical datasets. 

In [ ]:
# averaging all monthly data (1951-1990) together now. This takes awhile, track progress in command line!!
beginning = 1951
end = 1990

tmax_array = np.zeros(shape=(end-beginning,596,1385))
tmin_array = np.zeros(shape=(end-beginning,596,1385))
tavg_array = np.zeros(shape=(end-beginning,596,1385))
prcp_array = np.zeros(shape=(end-beginning,596,1385))

count = 0

for yr in tqdm(range(beginning, end)):
    resp = requests.get(f'https://noaa-nclimgrid-daily-pds.s3.amazonaws.com/v1-0-0/grids/{yr}/ncdd-{yr}{month_options.value}-grd-scaled.nc')
    file_name = f'ncdd-{yr}{month_options.value}-grd-scaled.nc'
    nc = netCDF4.Dataset(file_name, memory = resp.content)
    
    tmax_day = (nc.variables["tmax"])[day_value]
    tmin_day = (nc.variables["tmin"])[day_value]
    prcp_day = (nc.variables["prcp"])[day_value]
    tavg_day  = (nc.variables["tavg"])[day_value]
    
    nc.close()
    
    tmax_array[count,:,:] = tmax_day
    tmin_array[count,:,:] = tmin_day
    tavg_array[count,:,:] = tavg_day
    prcp_array[count,:,:] = prcp_day

    count += 1
    sleep(1)
    
#averaging array across all years
tmax_array_averaged = np.nanmean(tmax_array, axis=0)
tmin_array_averaged = np.nanmean(tmin_array, axis=0)
tavg_array_averaged = np.nanmean(tavg_array, axis=0)
prcp_array_averaged = np.nanmean(prcp_array, axis=0)

#plot
fig, axs = plt.subplots(nrows=2,ncols=2, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(16,9))

pc1 = axs[0,0].pcolormesh(lons, lats, tmax_array_averaged, cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc1, ax=axs[0,0],fraction=0.026, pad=0.04)
axs[0,0].set_title(f"Average maximum temperature on {month_options.value}/{day_options.value}\n from the record {beginning} to {end-1}")

pc2 = axs[0,1].pcolormesh(lons, lats, tmin_array_averaged, cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc2, ax=axs[0,1],fraction=0.026, pad=0.04)
axs[0,1].set_title(f"Average minimum temperature on {month_options.value}/{day_options.value}\n from the record {beginning} to {end-1}")

pc3 = axs[1,0].pcolormesh(lons, lats, tavg_array_averaged, cmap='jet', vmin=-30, vmax=50)
plt.colorbar(pc3, ax=axs[1,0],fraction=0.026, pad=0.04)
axs[1,0].set_title(f"Average temperature on {month_options.value}/{day_options.value}\n from the record {beginning} to {end-1}")

pc4 = axs[1,1].pcolormesh(lons, lats, prcp_array_averaged, cmap='jet', vmin=0, vmax=20)
plt.colorbar(pc4, ax=axs[1,1],fraction=0.026, pad=0.04)
axs[1,1].set_title(f"Average precipitation amount on {month_options.value}/{day_options.value}\n from the record {beginning} to {end-1}")

for i in range(0,2):
    for j in range(0,2):
        axs[i,j].add_feature(cf.LAKES)
        axs[i,j].add_feature(cf.OCEAN)
        states = cf.NaturalEarthFeature(
                category='cultural',
                name='admin_1_states_provinces_lines',
                scale='50m',
                facecolor='none')
        axs[i,j].add_feature(states, edgecolor="white")
        gl = axs[i,j].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', alpha=0.05, linestyle='-')
        gl.top_labels = False
        gl.left_labels = True
        gl.right_labels = False
        gl.xlines = True
        gl.ylines = True
plt.show(fig)
plt.close(fig)
plt.close("all")

### Step 5: Compare the variables from your selected day to the climatological average.
Goal: visualize whether your chosen day is warmer/colder, wetter/drier than typical conditions. 

In [ ]:
# departure from climatological mean
tmax_departure_from_mean = tmax[day_value] - tmax_array_averaged 
tmin_departure_from_mean = tmin[day_value] - tmin_array_averaged 
tavg_departure_from_mean = tavg[day_value] - tavg_array_averaged 
prcp_departure_from_mean = prcp[day_value] - prcp_array_averaged 

#plot
fig, axs = plt.subplots(nrows=2,ncols=2, subplot_kw={'projection': ccrs.PlateCarree()}, figsize=(16,9))

pc1 = axs[0,0].pcolormesh(lons, lats, tmax_departure_from_mean, cmap='RdBu_r', vmin=-20, vmax=20)
plt.colorbar(pc1, ax=axs[0,0],fraction=0.026, pad=0.04)
axs[0,0].set_title(f"Maximum temperature on {month_options.value}/{day_options.value}/{year_options.value}:\nDifference from the climatological average")

pc2 = axs[0,1].pcolormesh(lons, lats, tmin_departure_from_mean, cmap='RdBu_r', vmin=-20, vmax=20)
plt.colorbar(pc2, ax=axs[0,1],fraction=0.026, pad=0.04)
axs[0,1].set_title(f"Minimum temperature on {month_options.value}/{day_options.value}/{year_options.value}:\nDifference from the climatological average")

pc3 = axs[1,0].pcolormesh(lons, lats, tavg_departure_from_mean, cmap='RdBu_r', vmin=-20, vmax=20)
plt.colorbar(pc3, ax=axs[1,0],fraction=0.026, pad=0.04)
axs[1,0].set_title(f"Average temperature on {month_options.value}/{day_options.value}/{year_options.value}:\nDifference from the climatological average")

pc4 = axs[1,1].pcolormesh(lons, lats, prcp_departure_from_mean, cmap='RdBu', vmin=-20, vmax=20)
plt.colorbar(pc4, ax=axs[1,1],fraction=0.026, pad=0.04)
axs[1,1].set_title(f"Precipitation amount on {month_options.value}/{day_options.value}/{year_options.value}:\nDifference from the climatological average")

#universal formatting
for i in range(0,2):
    for j in range(0,2):
        axs[i,j].add_feature(cf.LAKES)
        axs[i,j].add_feature(cf.OCEAN)
        states = cf.NaturalEarthFeature(
                category='cultural',
                name='admin_1_states_provinces_lines',
                scale='50m',
                facecolor='none')
        axs[i,j].add_feature(states, edgecolor="white")
        gl = axs[i,j].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='black', alpha=0.05, linestyle='-')
        gl.top_labels = False
        gl.left_labels = True
        gl.right_labels = False
        gl.xlines = True
        gl.ylines = True
        
plt.show(fig)
plt.close(fig)
plt.close("all")

**NClimGrid documentation:**
- https://www.ncei.noaa.gov/products/land-based-station/nclimgrid-daily (NCEI product page)
- https://www.ncei.noaa.gov/data/nclimgrid-daily/doc/nclimgrid-daily_v1-0-0_user-guide.pdf (User guide)
- https://journals.ametsoc.org/view/journals/atot/39/12/JTECH-D-22-0024.1.xml (Journal article)

**CSP Access:**
- AWS: https://registry.opendata.aws/noaa-nclimgrid/
- Azure: https://microsoft.github.io/AIforEarthDataSets/data/noaa-nclimgrid.html

The unique component of this Jupyter notebook is that you are not requried to download any datasets -- all data will be pulled directly from the AWS cloud. You can learn more about NOAA's efforts to move more data to the cloud at this site: https://www.noaa.gov/information-technology/open-data-dissemination. As we continute to make more data widely accessible on the cloud, we'll also create more Jupyter notebooks like this one, so anyone can visualize weather and climate data without any cost or restriction. 